In [7]:
import numpy as np

In [8]:
def readInputData(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    data = {}
    
    # Parse material properties
    data['Material_Props'] = {}
    for i in range(len(lines)):
        if i < len(lines) - 1 and 'Material_Props:' in lines[i]:
            data['Material_Props']['Youngs_modulus'] = float(lines[i+1].split(':')[1])
            data['Material_Props']['Moment_of_inertia'] = float(lines[i+2].split(':')[1])
    
    # Parse number of nodes
    for i in range(len(lines)):
        if i < len(lines) and 'No._nodes:' in lines[i]:
            data['No._nodes'] = int(lines[i].split(':')[1])
    
    # Parse nodal coordinates
    data['Nodal_coords'] = []
    for i in range(len(lines)):
        if i < len(lines) and 'Nodal_coords:' in lines[i]:
            j = i + 1
            while j < len(lines) and lines[j].strip():
                data['Nodal_coords'].append(float(lines[j]))
                j += 1

    data['Nodal_coords'] = np.array(data['Nodal_coords'])
    
    # Parse element connectivity
    data['Element_connectivity'] = []
    for i in range(len(lines)):
        if i < len(lines) and 'Element_connectivity:' in lines[i]:
            j = i + 1
            while j < len(lines) and lines[j].strip():
                data['Element_connectivity'].append(list(map(int, lines[j].split())))
                j += 1

    data['Element_connectivity'] = np.array(data['Element_connectivity'])

    # Parse prescribed DOFs
    data['Prescribed_DOFs'] = []
    for i in range(len(lines)):
        if i < len(lines) and 'No._nodes_with_prescribed_DOFs:' in lines[i]:
            num_dofs = int(lines[i].split(':')[1])
            j = i + 2  # Skip "Node_#, DOF#, Value"
            for _ in range(num_dofs):
                if j < len(lines):
                    node_dof_value = list(map(float, lines[j].split()))
                    data['Prescribed_DOFs'].append(node_dof_value)
                    j += 1

    data['Prescribed_DOFs'] = np.array(data['Prescribed_DOFs'])
    
    # Parse spring stiffness
    data['Spring_stiffness'] = []
    for i in range(len(lines)):
        if i < len(lines) and 'No._nodes_with_spring_stiffness:' in lines[i]:
            num_springs = int(lines[i].split(':')[1])
            j = i + 2
            for _ in range(num_springs):
                if j < len(lines):
                    node_spring_value = list(map(float, lines[j].split()))
                    data['Spring_stiffness'].append(node_spring_value)
                    j += 1

    data['Spring_stiffness'] = np.array(data['Spring_stiffness'])

    # Parse prescribed loads
    data['Prescribed_loads'] = []
    for i in range(len(lines)):
        if i < len(lines) and 'No._nodes_with_prescribed_loads:' in lines[i]:
            num_loads = int(lines[i].split(':')[1])
            j = i + 2  # Skip "Node_#, DOF#, Traction_components"
            for _ in range(num_loads):
                if j < len(lines):
                    node_load_value = list(map(float, lines[j].split()))
                    data['Prescribed_loads'].append(node_load_value)
                    j += 1

    data['Prescribed_loads'] = np.array(data['Prescribed_loads'])

    # Parse body forces
    data['Body_forces'] = []
    for i in range(len(lines)):
        if i < len(lines) and 'No._prescribed_bodyforces:' in lines[i]:
            num_bodyforces = int(lines[i].split(':')[1])
            j = i + 1
            for _ in range(num_bodyforces):
                if j+1 < len(lines):
                    bodyforce_type = lines[j].split(':')[1].strip()
                    data['Body_forces'].append({'type': bodyforce_type, 'start_end_coords': [], 'val': []})
                    k = j + 2  # Skip "Coordinate, Value:"
                    while k < len(lines) and lines[k].strip() and lines[k].split()[0] != 'type:':
                        coord, val = tuple(map(float, lines[k].split(',')))
                        data['Body_forces'][-1]['start_end_coords'].append(coord)
                        data['Body_forces'][-1]['val'].append(val)
                        k += 1
                    
                    # Create lambda function for body force
                    if bodyforce_type == 'uniform':
                        data['Body_forces'][-1]['fn'] = lambda x: data['Body_forces'][-1]['val'][0]
                    elif bodyforce_type == 'linear':
                        x1, x2 = data['Body_forces'][-1]['start_end_coords']
                        f1, f2 = data['Body_forces'][-1]['val']
                        fn = lambda x: (f1 + (f2 - f1) * (x - x1) / (x2 - x1))
                        data['Body_forces'][-1]['fn'] = fn
                    else:
                        raise NotImplementedError("Only uniform and linear body forces are supported")
                    j = k

    return data

In [9]:
input_file = 'input_beam.txt'
data = readInputData(input_file)
data

{'Material_Props': {'Youngs_modulus': 210000000.0,
  'Moment_of_inertia': 0.0004},
 'No._nodes': 3,
 'Nodal_coords': array([ 0.,  6., 12.]),
 'Element_connectivity': array([[1, 2],
        [2, 3]]),
 'Prescribed_DOFs': array([[1., 1., 0.],
        [3., 1., 0.]]),
 'Spring_stiffness': array([[   2., 1000.]]),
 'Prescribed_loads': array([], dtype=float64),
 'Body_forces': [{'type': 'linear',
   'start_end_coords': [0.0, 6.0],
   'val': [10.0, 0.0],
   'fn': <function __main__.readInputData.<locals>.<lambda>(x)>},
  {'type': 'linear',
   'start_end_coords': [6.0, 12.0],
   'val': [0.0, 10.0],
   'fn': <function __main__.readInputData.<locals>.<lambda>(x)>}]}

In [10]:
E = data['Material_Props']['Youngs_modulus']
I = data['Material_Props']['Moment_of_inertia']

In [12]:
nen = len(data['Element_connectivity'][0])
print(nen)
ngp = nen
zeta, W = np.polynomial.legendre.leggauss(ngp)
zeta, W

2


(array([-0.57735027,  0.57735027]), array([1., 1.]))

In [11]:
def shape_fn(nen, zeta, le):
    if nen == 2:
        Nu1 = 0.25 * ((1-zeta)**2) * (2 + zeta)
        Nth1 = (le / 8) * ((1-zeta)**2) * (1+zeta)
        Nu2 = 0.25 * ((1+zeta)**2) * (2 - zeta)
        Nth2 = (le / 8) * ((1+zeta)**2) * (zeta-1)

        N = np.array([Nu1, Nth1, Nu2, Nth2])

        d2N = (1/le) * np.array([
            6*zeta/le,
             3*zeta - 1, 
             -6*zeta/le,
             3*zeta + 1])
        
        return N, d2N
    else:
        raise NotImplementedError("Only 2-noded elements are supported")

Stiffness matrix and force vector for 1st element of the beam

In [26]:
def get_kel(id_a, id_b, nen):
    kel = np.zeros((2*nen, 2*nen))
    fel = np.zeros((2*nen, 1))
    xae = data['Nodal_coords'][id_a]
    xbe = data['Nodal_coords'][id_b]

    le = xbe - xae
    for i in range(2*nen):
        for j in range(2*nen):
            val = 0.0
            for z, w in zip(zeta, W):
                temp = shape_fn(nen, z, le)
                d2Ni = temp[1][i] # Value of d2N_i / dx2 at zeta
                d2Nj = temp[1][j] # Value of d2N_j / dx2 at zeta
                val += d2Ni * d2Nj * w
            val *= (E*I*(le/2))
            kel[i, j] = val
    return kel,le

kel, le = get_kel(0, 1, 2)

correct_result = (E*I/(le**3))*np.array([
        [12, 6*le, -12, 6*le],
        [6*le, 4*le**2, -6*le, 2*le**2],
        [-12, -6*le, 12, -6*le],
        [6*le, 2*le**2, -6*le, 4*le**2]
    ]
)

print(kel)
print(correct_result)
print(np.allclose(kel, correct_result))

[[  4666.66666667  14000.          -4666.66666667  14000.        ]
 [ 14000.          56000.         -14000.          28000.        ]
 [ -4666.66666667 -14000.           4666.66666667 -14000.        ]
 [ 14000.          28000.         -14000.          56000.        ]]
[[  4666.66666667  14000.          -4666.66666667  14000.        ]
 [ 14000.          56000.         -14000.          28000.        ]
 [ -4666.66666667 -14000.           4666.66666667 -14000.        ]
 [ 14000.          28000.         -14000.          56000.        ]]
True


In [27]:
bodyforces = data['Body_forces']
def get_fel(bodyforces, id_a, id_b, nen):
    fel = np.zeros((2*nen, 1))
    xae = data['Nodal_coords'][id_a]
    xbe = data['Nodal_coords'][id_b]
    for q in bodyforces:
        coords = q['start_end_coords']
        fn = q['fn']
        x1, x2 = coords
        zeta_to_x = lambda zeta: ((x2 + x1)/2 + ((x2 - x1)/2)*zeta)
    # Check if [xae, xbe] overlaps with the body force coordinates
        if xae > x2 or xbe < x1:
            continue
        else:
        # Find the overlapping region
            x_start = max(xae, x1)
            x_end = min(xbe, x2)

            le = x_end - x_start

        # If le is close to 0, we can ignore the body force
            if abs(le) < 1e-10:
                continue

            for j in range(2*nen):
                val = 0.0
                for z, w in zip(zeta, W):
                    temp = shape_fn(nen, z, le)
                    Ni = temp[0][j]
                    val += Ni * fn(zeta_to_x(z)) * w
                val *= (le/2)
                fel[j] += val
    return fel

fel = get_fel(bodyforces, 0, 1, 2)

print(fel)

[[-21.66666667]
 [-20.        ]
 [ -8.33333333]
 [ 10.        ]]


In [28]:
print(nen)

2


In [29]:
nnodes = data["No._nodes"]

# data["Element_connectivity"]
K = np.zeros((2*nnodes, 2*nnodes))
F = np.zeros((2*nnodes, 1))

for i in range(len(data['Element_connectivity'])):
    elem = data['Element_connectivity'][i]
    elem = [x-1 for x in elem]  # Convert to 0-based indexing
    
    # kel    # local stiffness matrix, given
    # kel = np.array([[12, 6, -12, 6], [6, 4, -6, 2], [-12, -6, 12, -6], [6, 2, -6, 4]])
    kel = get_kel(elem[0], elem[1], nen)[0]
    fel = get_fel(bodyforces, elem[0], elem[1], nen)
    
    # Assemble
    for j in range(nen):
        for k in range(nen):
            K[2*elem[j], 2*elem[k]] += kel[2*j, 2*k]
            K[2*elem[j], 2*elem[k]+1] += kel[2*j, 2*k+1]
            K[2*elem[j]+1, 2*elem[k]] += kel[2*j+1, 2*k]
            K[2*elem[j]+1, 2*elem[k]+1] += kel[2*j+1, 2*k+1]

    # fel   # local force vector, given

    # Assemble F
    for j in range(nen):
        F[2*elem[j]] += fel[2*j]
        F[2*elem[j]+1] += fel[2*j+1]

print(K)
print(F)

[[  4666.66666667  14000.          -4666.66666667  14000.
       0.              0.        ]
 [ 14000.          56000.         -14000.          28000.
       0.              0.        ]
 [ -4666.66666667 -14000.           9333.33333333      0.
   -4666.66666667  14000.        ]
 [ 14000.          28000.              0.         112000.
  -14000.          28000.        ]
 [     0.              0.          -4666.66666667 -14000.
    4666.66666667 -14000.        ]
 [     0.              0.          14000.          28000.
  -14000.          56000.        ]]
[[-21.66666667]
 [-20.        ]
 [  0.        ]
 [ 20.        ]
 [ 21.66666667]
 [-20.        ]]


In [ ]:
[[  4666.66666667  14000.          -4666.66666667  14000.
       0.              0.        ]
 [ 14000.          56000.         -14000.          28000.
       0.              0.        ]
 [ -4666.66666667 -14000.           9333.33333333      0.
   -4666.66666667  14000.        ]
 [ 14000.          28000.              0.         112000.
  -14000.          28000.        ]
 [     0.              0.          -4666.66666667 -14000.
    4666.66666667 -14000.        ]
 [     0.              0.          14000.          28000.
  -14000.          56000.        ]]
[[-21.66666667]
 [-20.        ]
 [-30.        ]
 [-10.        ]
 [ -8.33333333]
 [ 10.        ]]